# Combining the static and dynamic methods for hypsometric relationships
In the first two notebooks, demonstrations were shown of how to derive hypsometric relationships through either static elevation datasets (notebook 01) or dynamic elevation from ICESat-2 (notebook 02). But why not use the strengths of both methods in case valid points are obtained for each method?

In this notebook we will load back the results and combine the results of both analyses into one hypsometric relationship. The points were pre-calculated so even if you did not run notebooks 01 and 02, you can run this notebook.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gwwstorage.static_hypso import Hypsometry
from scipy.interpolate import interp1d

### Load hypsometric points from the same reservoir, as derived from static and dynamic methods.
Let us first load the results back in memory so that we can investigate what these look like together

In [ ]:
static_fn = "datasets/static_hypso.csv"
dynamic_fn = "datasets/dynamic_hypso.csv"
df_static = pd.read_csv(static_fn)
df_dynamic = pd.read_csv(dynamic_fn)
ax = plt.subplot()
df_static.plot(y="water_level", x="surface_area", ax=ax, marker="x", linewidth=0., label="static (MERIT-Hydro)")
df_dynamic.plot(y="water_level", x="surface_area", ax=ax, marker="o", linewidth=0., label="dynamic (ICESat-2)")
ax.grid(linestyle="--")
ax.set_ylabel("Water level [m]")
ax.set_xlabel("Surface area [m2]")
ax.set_title("Hypsometric relationships Mita Hills -Zambia")


### Estimate offset between both methods
Ok, this is problematic. Both curves have roughly the same shape, but now the problem is that there are offsets between the two. These may be due to several reasons. Below we hypothesize on a few possible reasons
1. The elevation of the ICESat-2 altimetry points is related to the WGS84 ellipsoid according to the NASA documentation. However, the elevation of MERIT-Hydro is related to the EGM96 geoid. This creates offsets between the two that are not constant in place and work over the y-axis.
2. Our surface area calculations used to form elevation - surface area pairs (dot plot above) are perhaps overestimating surface area causing an offset on the x-axis.
3. There is other bias in either one of the methods causing some sort of offset.

Hence we need to make some assumptions in order to decide how to combine the two sets of points. We here assume the following:
1. Most of the bias is in the water level, hence we assume that differences in estimated surface area between the two methods are negligible.
2. A constant offset in water level may be assumed. This follows from the fact that the datums used for the static MERIT-Hydro and dynamic ICESat-2 are different.
3. At lower bathymetric levels (compared to the highest on record) the dynamic hypsometric points are more trustworthy as these are based on actual observations, while the static hyposmetry has to entirely rely on geomorphological assumptions.

There is no one size fits all solution to treating the combination. So we'll use some overlap between the two series to decide on the offset. To implement these assumptions we do the following:

First, we select a overlapping part of the points where we assume both methods give good estimates. This is likely in the upper part of the graph. Hence we select points that are above 50% of the range between lowest and highest surface area acoording to the static bathymetry points.


In [ ]:
# sort the dynamic points on surface area
area_thres = df_dynamic["surface_area"].min() + 0.5*(df_dynamic["surface_area"].max() - df_dynamic["surface_area"].min())
area_thres
df_dyn_overlap = df_dynamic[df_dynamic["surface_area"] > area_thres]
df_stat_overlap = df_static[df_static["surface_area"] > area_thres]


# df_dyn_sort
# df_static

We use the overlapping points to map the dynamic water levels to the static. We use a piece-wise interpolation function to map the values aginst each other and compute an average offset $H_{offset}$. This offset is used to make both series commensurate against each other. We compute this below and plot the results.


In [ ]:
# create interp function to match levels
f = interp1d(df_stat_overlap["surface_area"], df_stat_overlap["water_level"], fill_value="extrapolate")

# map dynamic against static
df_dyn_overlap["water_level_mapped"] = f(df_dyn_overlap["surface_area"])

# compute the average offset between the original and mapped water levels
h_offset = (df_dyn_overlap["water_level_mapped"] - df_dyn_overlap["water_level"]).mean()

# compute corrected water level
df_dynamic["water_level_corr"] = df_dynamic["water_level"] + h_offset

ax = plt.subplot()
df_stat_overlap.plot(y="water_level", x="surface_area", ax=ax, marker="x", linewidth=0., label="static (MERIT-Hydro)")
df_dynamic.plot(y="water_level_corr", x="surface_area", ax=ax, marker="o", linewidth=0., label="dynamic (ICESat-2), datum-corrected")
ax.grid(linestyle="--")
ax.set_ylabel("Water level [m]")
ax.set_xlabel("Surface area [m2]")
ax.set_title("Hypsometric relationships Mita Hills -Zambia")



### Make a new Hypsometry instance and show its behaviour

In [ ]:
water_level_combi = np.array(list(df_stat_overlap.water_level.values) + list(df_dynamic.water_level_corr.values))
surface_area_combi = np.array(list(df_stat_overlap.surface_area.values) + list(df_dynamic.surface_area.values))
hypso_combined = Hypsometry(
    water_level=water_level_combi,
    area=surface_area_combi
)
hypso_combined.plot()